In [1]:
import pandas as pd

In [12]:
import pandas as pd
import json
import os

# ----------------------------
# CONFIG
# ----------------------------
csv_file = "raw_data/Annotations.csv"   # CSV path
images_folder = "raw_data/images"              # folder with images
labels_folder = "./labels"              # folder to save YOLO labels
os.makedirs(labels_folder, exist_ok=True)

classes = ["Duckie", "Duckiebot", "QR code", "Intersection sign", "Signal sign"]

# ----------------------------
# LOAD CSV
# ----------------------------
df = pd.read_csv(csv_file)

# ----------------------------
# FUNCTION TO FIND IMAGE IN FOLDER
# ----------------------------
def find_image(filename, folder):
    # Try exact match first
    possible_path = os.path.join(folder, filename)
    if os.path.exists(possible_path):
        return possible_path
    # Try case-insensitive match
    for f in os.listdir(folder):
        if f.lower() == filename.lower():
            return os.path.join(folder, f)
    # Not found
    return None

# ----------------------------
# PROCESS EACH ROW
# ----------------------------
for idx, row in df.iterrows():
    original_name = row["original_filename"]
    img_path = find_image(original_name, images_folder)
    
    if img_path is None:
        print(f"Image not found for: {original_name}")
        continue

    # Parse JSON labels
    objects = json.loads(row["status"])
    lines = []
    for obj in objects:
        label = obj["label"]
        if label not in classes:
            continue
        class_id = classes.index(label)
        x_center = (obj["p1"]["x"] + obj["p2"]["x"]) / 2
        y_center = (obj["p1"]["y"] + obj["p2"]["y"]) / 2
        width = obj["p2"]["x"] - obj["p1"]["x"]
        height = obj["p2"]["y"] - obj["p1"]["y"]
        lines.append(f"{class_id} {x_center} {y_center} {width} {height}")

    # Save YOLO label file with the same name as image
    img_name_only = os.path.basename(img_path)
    label_file_path = os.path.join(labels_folder, os.path.splitext(img_name_only)[0] + ".txt")
    with open(label_file_path, "w") as f:
        f.write("\n".join(lines))

print("All labels created for YOLOv8!")


TypeError: string indices must be integers, not 'str'

In [15]:
import pandas as pd
import json
import os
import shutil
from sklearn.model_selection import train_test_split

# CONFIG
csv_file = "raw_data/Annotations.csv"
images_folder = "raw_data/images"
dataset_folder = "./dataset"
os.makedirs(dataset_folder, exist_ok=True)

classes = ["Duckie", "Duckiebot", "QR code", "Intersection sign", "Signal sign"]
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

def find_image(filename, folder):
    possible_path = os.path.join(folder, filename)
    if os.path.exists(possible_path):
        return possible_path
    for f in os.listdir(folder):
        if f.lower() == filename.lower():
            return os.path.join(folder, f)
    return None

for split in ["train", "val", "test"]:
    for sub in ["images", "labels"]:
        os.makedirs(os.path.join(dataset_folder, split, sub), exist_ok=True)

df = pd.read_csv(csv_file)
image_label_pairs = []

for idx, row in df.iterrows():
    original_name = row["original_filename"]
    img_path = find_image(original_name, images_folder)
    
    if img_path is None:
        print(f"Image not found for: {original_name}")
        continue

    # -----------------------------
    # SAFELY PARSE JSON
    # -----------------------------
    try:
        objects = json.loads(row["status"])
        if not isinstance(objects, list):
            objects = []
    except (json.JSONDecodeError, TypeError):
        objects = []

    if len(objects) == 0:
        print(f"No labels for image: {original_name}")
        continue

    lines = []
    for obj in objects:
        # skip if obj is not a dict
        if not isinstance(obj, dict):
            continue
        label = obj.get("label")
        if label not in classes:
            continue
        p1 = obj.get("p1")
        p2 = obj.get("p2")
        if p1 is None or p2 is None:
            continue

        class_id = classes.index(label)
        x_center = (p1["x"] + p2["x"]) / 2
        y_center = (p1["y"] + p2["y"]) / 2
        width = p2["x"] - p1["x"]
        height = p2["y"] - p1["y"]
        lines.append(f"{class_id} {x_center} {y_center} {width} {height}")

    if len(lines) == 0:
        continue  # skip images with no valid labels

    # Save label file
    img_name_only = os.path.basename(img_path)
    label_file_path = os.path.join(images_folder, os.path.splitext(img_name_only)[0] + ".txt")
    with open(label_file_path, "w") as f:
        f.write("\n".join(lines))

    image_label_pairs.append((img_path, label_file_path))

# SPLIT DATA AND CREATE YAML (same as before)
train_val, test = train_test_split(image_label_pairs, test_size=test_ratio, random_state=42)
train, val = train_test_split(train_val, test_size=val_ratio/(train_ratio + val_ratio), random_state=42)
splits = {"train": train, "val": val, "test": test}

for split_name, pairs in splits.items():
    for img_path, label_path in pairs:
        img_dest = os.path.join(dataset_folder, split_name, "images", os.path.basename(img_path))
        label_dest = os.path.join(dataset_folder, split_name, "labels", os.path.basename(label_path))
        shutil.copy(img_path, img_dest)
        shutil.copy(label_path, label_dest)

data_yaml = f"""
train: {os.path.join(dataset_folder, 'train', 'images')}
val: {os.path.join(dataset_folder, 'val', 'images')}
test: {os.path.join(dataset_folder, 'test', 'images')}

nc: {len(classes)}
names: {classes}
"""

with open(os.path.join(dataset_folder, "data.yaml"), "w") as f:
    f.write(data_yaml.strip())

print("Dataset ready for YOLOv8 training!")


No labels for image: B_BR_Duckbar_frame01214.jpg
No labels for image: B_BR_Duckbar_frame01311.jpg
No labels for image: b_br_yanberbot_3355.jpg
No labels for image: b_cr_yanberbot_0363.jpg
No labels for image: b_cr_yanberbot_1859.jpg
No labels for image: blue_br_yanberbot_1670.jpg
No labels for image: d_br_yanberbot_2683.jpg
No labels for image: D_CR_Duckbar_frame00258.jpg
No labels for image: D_CR_Duckbar_frame01191.jpg
No labels for image: d_cr_yanberbot_0397.jpg
No labels for image: d_cr_yanberbot_0560.jpg
No labels for image: d_cr_zgxbot_00042.jpg
No labels for image: d_cr_zgxbot_00052.jpg
No labels for image: Green_BR_Duckbar_frame00226.jpg
No labels for image: green_br_yanberbot_0470.jpg
No labels for image: green_br_yanberbot_2307.jpg
No labels for image: M_CR_Duckbar_frame00394.jpg
No labels for image: M_CR_Duckbar_frame01192.jpg
Dataset ready for YOLOv8 training!


In [8]:
# ...existing code...
# show number of columns and names
df.head()
# ...existing code...

,project_id,day,task_id,created_on,csv_upload_id,original_filename,file_id,status,task_units,base_customer_charge,total_customer_charge,download_url,media_width,media_height
0,10870,2018-11-27,a41710a6-f284-11e8-967e-2d58bfdba84d,Tue Nov 27 2018 20:40:19 GMT+0000 (UTC),a3acdd20-f284-11e8-967e-2d58bfdba84d,b_BR_doort_frame00355.jpg,31026ead-cb90-4ee0-b6fa-603734e90448,"[{""p1"":{""x"":0.2562368220357992,""y"":0.342564716...",1,0.2485,0.2485,https://hive-moderation.akamaized.net/31026ead...,640,480
1,10870,2018-11-27,a417ace1-f284-11e8-967e-2d58bfdba84d,Tue Nov 27 2018 20:40:19 GMT+0000 (UTC),a3acdd20-f284-11e8-967e-2d58bfdba84d,b_BR_doort_frame01343.jpg,3af6a718-8844-4923-bc70-08616c8c15e9,"[{""p1"":{""x"":0.5570447778735408,""y"":0.360642285...",1,0.2485,0.2485,https://hive-moderation.akamaized.net/3af6a718...,640,480
2,10870,2018-11-27,a41710a0-f284-11e8-967e-2d58bfdba84d,Tue Nov 27 2018 20:40:19 GMT+0000 (UTC),a3acdd20-f284-11e8-967e-2d58bfdba84d,b_BR_doort_frame00311.jpg,d6a9fc89-db70-411f-90fa-25680a786e4d,"[{""p1"":{""x"":0.13277708063471502,""y"":0.40155440...",1,0.2485,0.2485,https://hive-moderation.akamaized.net/d6a9fc89...,640,480
3,10870,2018-11-27,a41710a2-f284-11e8-967e-2d58bfdba84d,Tue Nov 27 2018 20:40:19 GMT+0000 (UTC),a3acdd20-f284-11e8-967e-2d58bfdba84d,b_BR_doort_frame00328.jpg,cf86c248-7b84-40ad-8235-208f15f12052,"[{""p1"":{""x"":0.13073216215380506,""y"":0.43269230...",1,0.2485,0.2485,https://hive-moderation.akamaized.net/cf86c248...,640,480
4,10870,2018-11-27,a4175ec2-f284-11e8-967e-2d58bfdba84d,Tue Nov 27 2018 20:40:19 GMT+0000 (UTC),a3acdd20-f284-11e8-967e-2d58bfdba84d,b_BR_doort_frame00596.jpg,31f68c5d-dd0a-451d-89d5-d4786c27ef1f,"[{""p1"":{""x"":0.055952380952380955,""y"":0.4952830...",1,0.2485,0.2485,https://hive-moderation.akamaized.net/31f68c5d...,640,480


In [2]:
from ultralytics import YOLO

# Load the model (best.pt in same folder)
model = YOLO("model/best.pt")

# Predict on an image
results = model("Screenshot 2026-01-14 at 12.35.02 PM.png", save=True, conf=0.25)

# Print detection results
for r in results:
    print(r.boxes)



image 1/1 /Users/abdullahzulfiqar/Desktop/Abdullah/freelancingwork/duckeytown/Screenshot 2026-01-14 at 12.35.02 PM.png: 640x640 1 QR code, 283.6ms
Speed: 3.5ms preprocess, 283.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict3
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([2.])
conf: tensor([0.5942])
data: tensor([[9.5571e+01, 7.0537e+02, 3.0213e+02, 8.1678e+02, 5.9423e-01, 2.0000e+00]])
id: None
is_track: False
orig_shape: (1274, 1284)
shape: torch.Size([1, 6])
xywh: tensor([[198.8517, 761.0755, 206.5616, 111.4061]])
xywhn: tensor([[0.1549, 0.5974, 0.1609, 0.0874]])
xyxy: tensor([[ 95.5709, 705.3724, 302.1325, 816.7786]])
xyxyn: tensor([[0.0744, 0.5537, 0.2353, 0.6411]])
